# Chapter 2: Coordinate Systems and Transformations

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** different coordinate frames used in navigation (LLH, ECEF, ENU)
2. **Convert** between geodetic (LLH) and Cartesian (ECEF) coordinates
3. **Transform** global coordinates to local East-North-Up (ENU) frames
4. **Work with** rotation representations (Euler angles, quaternions, rotation matrices)
5. **Apply** coordinate transformations to indoor positioning scenarios

### 📖 Book Reference

This notebook covers **Chapter 2: Coordinate Systems** with implementations of:
- **Eq. (2.1)**: LLH to ECEF transformation
- **Eq. (2.2)**: ECEF to LLH transformation
- **Eq. (2.3)**: ECEF to ENU transformation
- **Eq. (2.4)**: ENU to ECEF transformation
- **Eq. (2.5)-(2.10)**: Rotation representations (Euler, Quaternion, Matrix)

---


## 🚀 Setup

### Option A: Upload Repository ZIP (Recommended for Private Repos)

1. Download the repository as a ZIP file from your local machine
2. In Colab, click the folder icon on the left sidebar
3. Upload `IPIN_Book_Examples.zip` to `/content/`
4. Run the setup cell below

### Option B: Clone from GitHub (Public Repos)

If the repository is public, update the `GITHUB_REPO` variable below.


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 2: Coordinate Systems
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # e.g., "https://github.com/username/IPIN_Book_Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔍 Detecting setup method...")
    
    if os.path.exists('/content/IPIN_Book_Examples/core'):
        os.chdir('/content/IPIN_Book_Examples')
        print("✅ Repository already available.")
    elif os.path.exists('/content/IPIN_Book_Examples.zip'):
        print("📦 Found uploaded ZIP file, extracting...")
        import zipfile
        with zipfile.ZipFile('/content/IPIN_Book_Examples.zip', 'r') as zip_ref:
            zip_ref.extractall('/content/')
        if os.path.exists('/content/IPIN_Book_Examples-main'):
            os.rename('/content/IPIN_Book_Examples-main', '/content/IPIN_Book_Examples')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from ZIP complete!")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: Repository not found!")
        print("Upload 'IPIN_Book_Examples.zip' or set GITHUB_REPO variable.")
        raise FileNotFoundError("Repository not found.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import coordinate modules
from core.coords import (
    llh_to_ecef,
    ecef_to_llh,
    ecef_to_enu,
    enu_to_ecef,
    euler_to_rotation_matrix,
    rotation_matrix_to_euler,
    euler_to_quat,
    quat_to_euler,
    quat_to_rotation_matrix,
    rotation_matrix_to_quat,
)

print("\n" + "="*60)
print("✅ Setup complete! Coordinate modules loaded.")
print("="*60)


---

# Part 1: Coordinate Frames Overview

## 1.1 Common Coordinate Systems

| Frame | Description | Use Case |
|-------|-------------|----------|
| **LLH** | Latitude, Longitude, Height (geodetic) | GPS coordinates, global positioning |
| **ECEF** | Earth-Centered Earth-Fixed (Cartesian) | Satellite navigation, global computations |
| **ENU** | East-North-Up (local tangent) | Local positioning, indoor navigation |
| **NED** | North-East-Down | Aviation, robotics |
| **Body** | Attached to vehicle/sensor | IMU measurements, sensor fusion |

### WGS84 Ellipsoid Constants

- Semi-major axis: **a = 6,378,137 m**
- Flattening: **f = 1/298.257223563**
- Semi-minor axis: **b = 6,356,752.314 m**


## 1.2 Example: LLH to ECEF Transformation (Eq. 2.1)

Convert geodetic coordinates (latitude, longitude, height) to Earth-Centered Earth-Fixed Cartesian coordinates.


In [ ]:
# Example 1: LLH to ECEF Transformation
print("="*70)
print("Example 1: LLH to ECEF Transformation (Eq. 2.1)")
print("="*70)

# Define location: San Francisco (37.7749°N, 122.4194°W)
lat_sf = np.deg2rad(37.7749)
lon_sf = np.deg2rad(-122.4194)
height_sf = 0.0  # Sea level

print(f"\n📍 Location: San Francisco")
print(f"  Latitude:  {np.rad2deg(lat_sf):.4f}° N")
print(f"  Longitude: {np.rad2deg(lon_sf):.4f}° W")
print(f"  Height:    {height_sf:.1f} m (above WGS84 ellipsoid)")

# Convert to ECEF
xyz_sf = llh_to_ecef(lat_sf, lon_sf, height_sf)

print(f"\n🌍 ECEF Coordinates:")
print(f"  X: {xyz_sf[0]:>15,.2f} m")
print(f"  Y: {xyz_sf[1]:>15,.2f} m")
print(f"  Z: {xyz_sf[2]:>15,.2f} m")

# Verify: distance from Earth center
distance_from_center = np.linalg.norm(xyz_sf)
print(f"\n  Distance from Earth center: {distance_from_center/1000:,.2f} km")
print(f"  (Earth radius ~6,371 km)")


In [ ]:
# Example 2: Round-trip conversion (LLH -> ECEF -> LLH)
print("="*70)
print("Example 2: Round-Trip Conversion (Eq. 2.1 + Eq. 2.2)")
print("="*70)

# Convert back to LLH
llh_recovered = ecef_to_llh(xyz_sf[0], xyz_sf[1], xyz_sf[2])

print(f"\nOriginal LLH:")
print(f"  Latitude:  {np.rad2deg(lat_sf):.10f}°")
print(f"  Longitude: {np.rad2deg(lon_sf):.10f}°")
print(f"  Height:    {height_sf:.10f} m")

print(f"\nRecovered LLH:")
print(f"  Latitude:  {np.rad2deg(llh_recovered[0]):.10f}°")
print(f"  Longitude: {np.rad2deg(llh_recovered[1]):.10f}°")
print(f"  Height:    {llh_recovered[2]:.10f} m")

# Compute errors
lat_error = np.abs(llh_recovered[0] - lat_sf)
lon_error = np.abs(llh_recovered[1] - lon_sf)
h_error = np.abs(llh_recovered[2] - height_sf)

print(f"\n✅ Round-trip errors:")
print(f"  Latitude error:  {lat_error:.2e} rad = {lat_error * 6371000:.2e} m")
print(f"  Longitude error: {lon_error:.2e} rad = {lon_error * 6371000 * np.cos(lat_sf):.2e} m")
print(f"  Height error:    {h_error:.2e} m")
print(f"\n💡 Sub-nanometer accuracy achieved!")


---

# Part 2: Local ENU Frame

## 2.1 Why ENU?

For **indoor positioning**, we work in a **local tangent plane** centered at a reference point (e.g., building entrance):

- **East (E)**: Positive towards geographic East
- **North (N)**: Positive towards geographic North  
- **Up (U)**: Positive away from Earth center

This provides intuitive coordinates in meters for indoor applications.


In [ ]:
# Example 3: Local ENU Frame (Eq. 2.3, 2.4)
print("="*70)
print("Example 3: Local ENU Frame Transformation")
print("="*70)

# Define reference point (building entrance)
lat_ref = np.deg2rad(37.7749)
lon_ref = np.deg2rad(-122.4194)
height_ref = 0.0

print(f"\n🏢 Reference point (building entrance):")
print(f"  Latitude:  {np.rad2deg(lat_ref):.4f}° N")
print(f"  Longitude: {np.rad2deg(lon_ref):.4f}° W")

# Define target points
targets = [
    ("100m East", lat_ref, lon_ref + np.deg2rad(100/78800), 0.0),
    ("100m North", lat_ref + np.deg2rad(100/111000), lon_ref, 0.0),
    ("50m Up", lat_ref, lon_ref, 50.0),
]

print(f"\n📍 Target points in ENU coordinates:")
for name, lat_tgt, lon_tgt, h_tgt in targets:
    # Convert target LLH to ECEF
    xyz_tgt = llh_to_ecef(lat_tgt, lon_tgt, h_tgt)
    
    # Convert ECEF to ENU (relative to reference)
    enu = ecef_to_enu(xyz_tgt[0], xyz_tgt[1], xyz_tgt[2], lat_ref, lon_ref, height_ref)
    
    print(f"\n  {name}:")
    print(f"    ENU: E={enu[0]:>8.2f}m, N={enu[1]:>8.2f}m, U={enu[2]:>8.2f}m")


In [ ]:
# Example 4: Practical Indoor Positioning Scenario
print("="*70)
print("Example 4: Indoor Positioning Scenario")
print("="*70)

print(f"\n🏢 Building entrance (reference): {np.rad2deg(lat_ref):.4f}°N, {np.rad2deg(lon_ref):.4f}°W")

# Define user locations in ENU (local coordinates)
user_positions = [
    ("Lobby (entrance)", np.array([0.0, 0.0, 0.0])),
    ("Room 101", np.array([15.0, 10.0, 0.0])),
    ("Room 201 (2nd floor)", np.array([15.0, 10.0, 3.5])),
    ("Parking (basement)", np.array([-5.0, -20.0, -2.5])),
]

print("\n📍 User locations:")
for name, enu_pos in user_positions:
    # Convert ENU back to ECEF
    xyz = enu_to_ecef(enu_pos[0], enu_pos[1], enu_pos[2], lat_ref, lon_ref, height_ref)
    
    # Convert ECEF to LLH (for GPS coordinates)
    llh = ecef_to_llh(xyz[0], xyz[1], xyz[2])
    
    print(f"\n  {name}:")
    print(f"    ENU: E={enu_pos[0]:>6.1f}m, N={enu_pos[1]:>6.1f}m, U={enu_pos[2]:>6.1f}m")
    print(f"    LLH: {np.rad2deg(llh[0]):.6f}°, {np.rad2deg(llh[1]):.6f}°, {llh[2]:.2f}m")


---

# Part 3: Rotation Representations

## 3.1 Overview

Three common ways to represent 3D rotations:

| Representation | Parameters | Pros | Cons |
|----------------|-----------|------|------|
| **Euler Angles** | 3 (roll, pitch, yaw) | Intuitive, human-readable | Gimbal lock at ±90° pitch |
| **Rotation Matrix** | 9 (3×3) | Direct transformation | 6 redundant parameters |
| **Quaternion** | 4 (w, x, y, z) | No gimbal lock, smooth interpolation | Less intuitive |

**Convention used:** ZYX Euler angles (yaw → pitch → roll)


In [ ]:
# Example 5: Rotation Representations (Eq. 2.5-2.10)
print("="*70)
print("Example 5: Rotation Representations")
print("="*70)

# Define Euler angles (roll, pitch, yaw)
roll = np.deg2rad(10.0)   # 10° roll (rotation about x-axis)
pitch = np.deg2rad(20.0)  # 20° pitch (rotation about y-axis)
yaw = np.deg2rad(30.0)    # 30° yaw (rotation about z-axis)

print(f"\n🎯 Euler Angles (ZYX convention):")
print(f"  Roll:  {np.rad2deg(roll):.1f}°")
print(f"  Pitch: {np.rad2deg(pitch):.1f}°")
print(f"  Yaw:   {np.rad2deg(yaw):.1f}°")

# Convert to rotation matrix
R = euler_to_rotation_matrix(roll, pitch, yaw)
print(f"\n📐 Rotation Matrix (Eq. 2.5):")
print(f"{R}")
print(f"  Determinant: {np.linalg.det(R):.6f} (should be 1.0)")
print(f"  Orthogonal check (R'R): {np.allclose(R.T @ R, np.eye(3))}")

# Convert to quaternion
q = euler_to_quat(roll, pitch, yaw)
print(f"\n🔄 Quaternion [w, x, y, z] (Eq. 2.7):")
print(f"  [{q[0]:.6f}, {q[1]:.6f}, {q[2]:.6f}, {q[3]:.6f}]")
print(f"  Norm: {np.linalg.norm(q):.6f} (should be 1.0)")


In [ ]:
# Example 6: Applying Rotation to Vectors
print("="*70)
print("Example 6: Applying Rotation to Vectors")
print("="*70)

# Vector in body frame (e.g., sensor pointing forward)
v_body = np.array([1.0, 0.0, 0.0])
print(f"\n📍 Vector in body frame: {v_body}")
print(f"  (Sensor pointing forward)")

# Rotate to navigation frame using matrix
R_from_quat = quat_to_rotation_matrix(q)
v_nav = R_from_quat @ v_body
print(f"\n🧭 Vector in navigation frame: [{v_nav[0]:.4f}, {v_nav[1]:.4f}, {v_nav[2]:.4f}]")

# Round-trip conversion test
print("\n" + "="*70)
print("Example 7: Round-Trip Rotation Conversions")
print("="*70)

# Euler -> Matrix -> Euler
euler_recovered = rotation_matrix_to_euler(R)
print(f"\nEuler → Matrix → Euler:")
print(f"  Original: [{np.rad2deg(roll):.1f}°, {np.rad2deg(pitch):.1f}°, {np.rad2deg(yaw):.1f}°]")
print(f"  Recovered: [{np.rad2deg(euler_recovered[0]):.1f}°, {np.rad2deg(euler_recovered[1]):.1f}°, {np.rad2deg(euler_recovered[2]):.1f}°]")

# Quaternion -> Matrix -> Quaternion
q_recovered = rotation_matrix_to_quat(R_from_quat)
print(f"\nQuaternion → Matrix → Quaternion:")
print(f"  Original:  [{q[0]:.6f}, {q[1]:.6f}, {q[2]:.6f}, {q[3]:.6f}]")
print(f"  Recovered: [{q_recovered[0]:.6f}, {q_recovered[1]:.6f}, {q_recovered[2]:.6f}, {q_recovered[3]:.6f}]")


---

# Summary

## Key Takeaways

1. **LLH ↔ ECEF** (Eq. 2.1-2.2): Convert between GPS coordinates and global Cartesian
   - Sub-nanometer round-trip accuracy
   - Foundation for all positioning systems

2. **ECEF ↔ ENU** (Eq. 2.3-2.4): Create local coordinate frames for indoor positioning
   - Intuitive meters-based coordinates
   - Essential for building-scale navigation

3. **Rotation Representations** (Eq. 2.5-2.10):
   - **Euler angles**: Human-readable (use for display)
   - **Quaternions**: Computation (avoid gimbal lock)
   - **Matrices**: Direct vector transformation

## Best Practices

| Task | Recommended Representation |
|------|---------------------------|
| User display | Euler angles (degrees) |
| Sensor fusion | Quaternions |
| Vector transformation | Rotation matrices |
| Data storage | Quaternions (compact, normalized) |

---

**Next Steps:** Apply these concepts in Chapter 3 (State Estimation), Chapter 6 (Dead Reckoning), and Chapter 8 (Sensor Fusion)!
